In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Row
from pyspark.sql.window import Window
from pyspark.sql.types import NumericType
from graphframes import GraphFrame
from pyiceberg.catalog import load_catalog
import re

# --- 1. Spark с Iceberg + MinIO ---
spark = SparkSession.getActiveSession()
if spark is not None:
    print("✅ Подключен к существующей SparkSession")
else:
    print("⚠️ Активная сессия не найдена — создаю новую...")
    spark = (
        SparkSession.builder
            .appName("glazkov_type_rest")
            .getOrCreate()
    )
print("=" * 80)

# 🧠 Основные параметры кластера
print(f"Spark master:            {spark.sparkContext.master}")
print(f"Spark version:           {spark.version}")
print(f"App name:                {spark.sparkContext.appName}")
print(f"Executors (parallelism): {spark.sparkContext.defaultParallelism}")
print(f"Executor memory:         {spark.conf.get('spark.executor.memory', 'n/a')}")
print(f"Driver memory:           {spark.conf.get('spark.driver.memory', 'n/a')}")

print("-" * 80)

# 🧊 Iceberg / Catalog / Warehouse
print(f"Catalog type:            {spark.conf.get('spark.sql.catalog.ice.type', 'n/a')}")
print(f"Catalog URI:             {spark.conf.get('spark.sql.catalog.ice.uri', 'n/a')}")
print(f"Warehouse:               {spark.conf.get('spark.sql.catalog.ice.warehouse', 'n/a')}")
print(f"Default catalog:         {spark.conf.get('spark.sql.defaultCatalog', 'n/a')}")

print("-" * 80)

# 🪣 S3 / MinIO настройки
print(f"S3A endpoint:            {spark.conf.get('spark.hadoop.fs.s3a.endpoint', 'n/a')}")
print(f"S3A path style:          {spark.conf.get('spark.hadoop.fs.s3a.path.style.access', 'n/a')}")
print(f"S3A SSL enabled:         {spark.conf.get('spark.hadoop.fs.s3a.connection.ssl.enabled', 'n/a')}")
print(f"S3A provider:            {spark.conf.get('spark.hadoop.fs.s3a.aws.credentials.provider', 'n/a')}")

print("-" * 80)

# ⚙️ Iceberg расширения и классы
print(f"Iceberg extensions:      {spark.conf.get('spark.sql.extensions', 'n/a')}")
print(f"Iceberg catalog class:   {spark.conf.get('spark.sql.catalog.ice', 'n/a')}")

print("-" * 80)

# 🧾 Проверим доступность REST-каталога
try:
    from urllib.request import urlopen
    uri = spark.conf.get("spark.sql.catalog.ice.uri")
    resp = urlopen(uri + "/v1/config").read().decode("utf-8")
    print(f"REST catalog reachable ✅ {uri}")
except Exception as e:
    print(f"⚠️ REST catalog NOT reachable ({e})")

print("=" * 80)

# --- Подключаем PostgreSQL ---
# pg_url = "jdbc:postgresql://192.168.150.10:5432/MZ_DWH"
# pg_props = {
#     "user": "glazole",
#     "password": "MEbt7xr39yRhXUBj",
#     "driver": "org.postgresql.Driver"
# }

✅ Подключен к существующей SparkSession
Spark master:            spark://spark-master:7077
Spark version:           3.5.0
App name:                glazkov_type_rest_graph_frame
Executors (parallelism): 2
Executor memory:         2g
Driver memory:           3g
--------------------------------------------------------------------------------
Catalog type:            rest
Catalog URI:             http://iceberg-rest:8181
Warehouse:               s3a://warehouse/iceberg/
Default catalog:         ice
--------------------------------------------------------------------------------
S3A endpoint:            http://minio:9000
S3A path style:          true
S3A SSL enabled:         false
S3A provider:            n/a
--------------------------------------------------------------------------------
Iceberg extensions:      org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
Iceberg catalog class:   org.apache.iceberg.spark.SparkCatalog
---------------------------------------------------

In [5]:
spark.sql("SHOW NAMESPACES IN ice").show()

+---------+
|namespace|
+---------+
+---------+



In [6]:
cat = spark.catalog.listDatabases()
print(cat)

[]


In [ ]:
# ДЛЯ НЕСКОЛЬКИХ ТАБЛИЦ
namespace = "bronze"
tables = [  # Используем список вместо множества
    ("f1_manuf_import", "v1"),
    ("f2_out", "v32"),
    ("f2_out_agg_16_09_2025", "v1"),
    ("f3_residues", "v5"),
    ("f3_residues_agg_16_09_2025", "v1"),
    ("f4_sprav_gtin", "v1"),
    ("f5_sprav_md", "v1"),
    ("f6_sprav_players", "v1"),
    ("f7_sprav_gtin_vac", "v1"),
    ("gtin_unique_stg", "v1"),
    ("main_spr", "v1")
]

for table_name, version in tables:
    try:
        spark.sql(f"DROP TABLE IF EXISTS {namespace}.{table_name}")
        spark.sql(f"""
            CALL ice.system.register_table(
                '{namespace}.{table_name}',
                's3a://crpt/warehouse/iceberg/{namespace}/{table_name}/metadata/{version}.metadata.json'
            )
        """)
        print(f"Таблица {table_name} успешно зарегистрирована")
    except Exception as e:
        print(f"Ошибка при регистрации таблицы {table_name}: {str(e)}")

In [4]:
cat = load_catalog("ice", uri="http://iceberg-rest:8181")
cat.list_tables("bronze")

NoSuchNamespaceError: NoSuchNamespaceException: Namespace does not exist: bronze

In [8]:
spark.stop()